In [28]:
pip install pandas sqlalchemy mysql-connector-python openpyxl

     -------------------------------------- 250.9/250.9 KB 3.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ASUS\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
#Librerías necesarias
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import csv
import os
from bs4 import BeautifulSoup
from docx import Document
from sqlalchemy import create_engine

In [ ]:
############################ WEBSCRAPPING FALABELLA ############################

In [ ]:
# Configuración para Selenium
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--headless")  # Ejecutar en modo headless
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36"
)
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

# Ruta del driver
driver_path = 'D:\\Proyecto Web Scrapping\\chromedriver.exe'  # Cambia esta ruta si es necesario
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Lista de productos (enlaces y modelos)
F_productos = [
    # Xiaomi
    ("https://www.falabella.com.co/falabella-co/product/127913189/Celular-Xiaomi-Poco-X6-Pro-5G-512Gb-12Ram-64Mp-Negro/127913190", "Xiaomi", "Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp", 1899900),
    ("https://www.falabella.com.co/falabella-co/product/127913775/Celular-Xiaomi-Poco-X6-Pro-5G-512Gb-12Ram-64Mp-Amarillo/127913776", "Xiaomi", "Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp", 1449900),
    ("https://www.falabella.com.co/falabella-co/product/127916553/Celular-Xiaomi-Redmi-Note-13-Pro-Plus-5G-512Gb-12Ram-200Mp-Negro/127916554", "Xiaomi", "Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp", 2199900),
    ("https://www.falabella.com.co/falabella-co/product/128104744/Celular-Xiaomi-Redmi-Note-13-PRO-PLUS-5G-512GB-12GB-RAM-Negro/128104745", "Xiaomi", "Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp", 2199900),
    ("https://www.falabella.com.co/falabella-co/product/130055074/Celular-Xiaomi-Redmi-Note-13-Pro-Plus-5g-512gb-12ram-PURPURA/130055075", "Xiaomi", "Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp",3099700),
    ("https://www.falabella.com.co/falabella-co/product/127687136/CELULAR-XIAOMI-REDMI-13C-256GB-8GB-RAM-NEGRO/127687137", "Xiaomi", "XIAOMI REDMI 13C 256GB 8GB RAM", 600000),
    ("https://www.falabella.com.co/falabella-co/product/128261646/Celular-Xiaomi-Redmi-13C-256GB-8RAM-Negro/128261647", "Xiaomi", "XIAOMI REDMI 13C 256GB 8GB RAM", 600000),
    ("https://www.falabella.com.co/falabella-co/product/128128076/Xiaomi-Redmi-Note-13-Pro-4g-256gb-8gb-Ram-Negro/128128077", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram", 879900),
    ("https://www.falabella.com.co/falabella-co/product/128807587/Xiaomi-Redmi-Note-13-Pro-4g-256gb-8gb-Ram-Purpura/128807589", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram", 879900),
    ("https://www.falabella.com.co/falabella-co/product/prod13530339/Celular-Xiaomi-Redmi-Note-13-Pro-256GB-8GB-Camara-Posterior-200MP+8MP+2MP-Camara-Frontal-16MP-Pantalla-6.67-Pulgadas/72853341", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram", 949900),
    ("https://www.falabella.com.co/falabella-co/product/128128195/Xiaomi-Redmi-Note-13-Pro-4g-256gb-8gb-Ram-Forest-Green/128128196", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram", 1299900),
    ("https://www.falabella.com.co/falabella-co/product/128931528/Celular-Xiaomi-Redmi-Note-13-8GB-256GB-Negro/128931529", "Xiaomi", "Xiaomi Redmi Note 13 8GB 256GB", 1079900),
    ("https://www.falabella.com.co/falabella-co/product/127891485/Celular-Xiaomi-Redmi-Note-13-256GB-8GB-RAM-Negro/127891486", "Xiaomi", "Xiaomi Redmi Note 13 8GB 256GB", 669900),
    
    # Samsung
    ("https://www.falabella.com.co/falabella-co/product/129586393/CELULAR-SAMSUNG-GALAXY-A55-5G-256GB-8-RAM-AZUL-NAVY/129586394", "Samsung", "SAMSUNG GALAXY A55", 2499900),
    ("https://www.falabella.com.co/falabella-co/product/129554425/Celular-Samsung-Galaxy-A55-5G-256Gb-8Ram-Azul-Oscuro/129554428", "Samsung", "SAMSUNG GALAXY A55", 2000000),
    ("https://www.falabella.com.co/falabella-co/product/prod13530496/Celular-Samsung-A15-256GB-8GB-RAM-Camara-Posterior-50-MP-Camara-Frontal-12MP-Pantalla-6.5-Pulgadas-+-MediaTek-MT6789V-CD-/72854633", "Samsung", "Samsung A15 256GB", 699900),
    ("https://www.falabella.com.co/falabella-co/product/132236410/CELULAR-SAMSUNG-GALAXY-A15-8+256GB-AZUL-NEGRO/132236411", "Samsung", "Samsung A15 256GB", 612900),
    ("https://www.falabella.com.co/falabella-co/product/128004219/CELULAR-SAMSUNG-GALAXY-A15-256GB-8-RAM-NEGRO-(Blue-Black)/128004220", "Samsung", "Samsung A15 256GB",599900 ),
    ("https://www.falabella.com.co/falabella-co/product/132236494/CELULAR-SAMSUNG-GALAXY-A15-8+256GB-AZUL/132236495", "Samsung", "Samsung A15 256GB", 612900),
    ("https://www.falabella.com.co/falabella-co/product/128906319/Celular-Samsung-Galaxy-A25-5G-256GB-8RAM+-Cargador/128906321", "Samsung", "Samsung Galaxy A25 5G 256GB 8RAM", 1235000),
    ("https://www.falabella.com.co/falabella-co/product/131931272/Celular-SAMSUNG-GALAXY-A25-256GB-+-8-RAM-AZUL-CLARO/131931273", "Samsung", "Samsung Galaxy A25 5G 256GB 8RAM", 1199900),
    ("https://www.falabella.com.co/falabella-co/product/72899217/Celular-Samsung-Galaxy-A35-5G-256GB-8GB-RAM-camara-posterior-50-MP-camara-frontal-13MP-pantalla-6.6-Pulgadas-+-Exynos-1380/72899217", "Samsung", "Samsung Galaxy A35 5G 256GB 8GB RAM", 969900),
    ("https://www.falabella.com.co/falabella-co/product/133744591/Celular-Samsung-Galaxy-A35-256Gb-5g-Lila/133744592", "Samsung", "Samsung Galaxy A35 5G 256GB 8GB RAM", 969900),
    ("https://www.falabella.com.co/falabella-co/product/128668464/Celular-Samsung-Galaxy-A05S-128Gb-Negro/128668465", "Samsung", "Samsung Galaxy A05S 128Gb", 449900),
    ("https://www.falabella.com.co/falabella-co/product/128035979/Samsung-S24-Ultra-256GB-12GB-Violet/128035980", "Samsung", "Samsung S24 Ultra 256GB 12GB", 7500000),
    ("https://www.falabella.com.co/falabella-co/product/128853893/Celular-Samsung-Galaxy-S24-Ultra-De-256GB12GB-RAM-Gris-5G/128853894", "Samsung", "Samsung S24 Ultra 256GB 12GB", 7500000),
    ("https://www.falabella.com.co/falabella-co/product/129574090/Samsung-Galaxy-S24-Ultra-512GB-12GB-Ram-5G-Violeta/129574091", "Samsung", "Samsung Galaxy S24 Ultra 512GB 12GB", 7499990),

    #Apple
    ("https://www.falabella.com.co/falabella-co/product/prod12000048/iPhone-13-128-GB-5G-4GB-RAM-Pantalla-6.1-Pulgadas-Chip-A15-Bionic/19604508", "Apple", "iPhone 13 128 GB 5G 4GB RAM", 2799900),
    ("https://www.falabella.com.co/falabella-co/product/prod13430667/iPhone-15-de-128GB-5G-6GB-RAM-Pantalla-6.1-pulgadas-Chip-A16-Bionic-Carga-Tipo-C-Dynamic-Island-Camara-48MP/72752105", "Apple", "iPhone 15 128GB 5G 6GB RAM", 3759900),
    ("https://www.falabella.com.co/falabella-co/product/prod13070047/iPhone-14-128GB-5G-6GB-RAM-Pantalla-6.1-Pulgadas-Chip-A15-Bionic-/60999138", "Apple", "iPhone 14 128GB 5G 6GB RAM", 3199900),
    ("https://www.falabella.com.co/falabella-co/product/prod10900027/iPhone-11-128-GB-4GB-RAM-Pantalla-6.1-Pulgadas-Chip-A13-Bionic-Camara-12-MP/9722720", "Apple", "iPhone 11 128 GB 4GB RAM", 2549900),
]

# Lista para almacenar los datos
F_data = []

# Iterar sobre cada producto
for url, marca, modelo in F_productos:
    print(f"Procesando: {modelo} ({marca})")

    # Cargar la página del producto
    driver.get(url)

    # Esperar a que se cargue el contenedor de comentarios
    try:
        wait = WebDriverWait(driver, 25)  # Aumentar si es necesario para conexiones lentas
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='user-reviews-container']")))

        # Cargar todos los comentarios
        comentarios = []
        while True:
            try:
                ver_mas_btn = driver.find_element(By.ID, "more-comments-button")
                driver.execute_script("arguments[0].scrollIntoView();", ver_mas_btn)
                ver_mas_btn.click()

                # Esperar dinámicamente hasta que se carguen nuevos comentarios
                WebDriverWait(driver, 25).until(
                    lambda d: len(d.find_elements(By.CSS_SELECTOR, "div._review-text_16yc3_2")) > len(comentarios)
                )

                # Actualizar el número de comentarios cargados
                comentarios = driver.find_elements(By.CSS_SELECTOR, "div._review-text_16yc3_2")
                print(f"Comentarios cargados hasta ahora: {len(comentarios)}")  # Log de progreso

            except Exception:
                print("No hay más comentarios para cargar.")
                break

        # Extraer los comentarios y calificaciones
        F_reviews = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='review']")
        for review in F_reviews:
            try:
                # Extraer el comentario
                comentario_element = review.find_element(By.CSS_SELECTOR, "div._review-text_16yc3_2")
                comentario = comentario_element.text.strip()

                # Buscar las estrellas dentro del contenedor actual
                estrellas_llenas = review.find_elements(By.CSS_SELECTOR, "span._star-full_10acl_10")
                calificacion = len(estrellas_llenas)  # Contar las estrellas llenas

                print(f"Comentario: {comentario[:30]}... | Calificación: {calificacion}")  # Log para depuración

                # Guardar en la lista
                F_data.append({"comentario": comentario, "calificacion": calificacion, "marca": marca, "modelo": modelo})

            except Exception as e:
                print(f"Error procesando una review: {e}")

    except Exception as e:
        print(f"Error procesando {modelo}: {e}")

# Crear un DataFrame con los resultados
F_df = pd.DataFrame(F_data)

# Exportar a CSV
F_df.to_csv("comentarios_productos_falabella.csv", index=False, encoding="utf-8")
print(f"Se procesaron {len(F_data)} comentarios en total. Archivo guardado como 'comentarios_productos_falabella.csv'.")

# Cerrar el navegador
driver.quit()

Comentario 1: Está bueno y funciona bien, esperemos que no salga chimbo, como mi ex.
Comentario 2: Me parece un movil muy bonito, tiene un duracion de bateria excelente.


In [ ]:
# Configuración para Selenium
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--headless")  # Ejecutar en modo headless
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36"
)
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

# Ruta del driver
driver_path = 'D:\\Proyecto Web Scrapping\\chromedriver.exe'  # Cambia esta ruta si es necesario
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Lista de productos (enlaces y modelos)
F_productos = [
    # Xiaomi
    ("https://www.falabella.com.co/falabella-co/product/127913189/Celular-Xiaomi-Poco-X6-Pro-5G-512Gb-12Ram-64Mp-Negro/127913190", "Xiaomi", "Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp", 1899900),
    ("https://www.falabella.com.co/falabella-co/product/127913775/Celular-Xiaomi-Poco-X6-Pro-5G-512Gb-12Ram-64Mp-Amarillo/127913776", "Xiaomi", "Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp", 1449900),
    ("https://www.falabella.com.co/falabella-co/product/127916553/Celular-Xiaomi-Redmi-Note-13-Pro-Plus-5G-512Gb-12Ram-200Mp-Negro/127916554", "Xiaomi", "Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp", 2199900),
    ("https://www.falabella.com.co/falabella-co/product/128104744/Celular-Xiaomi-Redmi-Note-13-PRO-PLUS-5G-512GB-12GB-RAM-Negro/128104745", "Xiaomi", "Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp", 2199900),
    ("https://www.falabella.com.co/falabella-co/product/130055074/Celular-Xiaomi-Redmi-Note-13-Pro-Plus-5g-512gb-12ram-PURPURA/130055075", "Xiaomi", "Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp",3099700),
    ("https://www.falabella.com.co/falabella-co/product/127687136/CELULAR-XIAOMI-REDMI-13C-256GB-8GB-RAM-NEGRO/127687137", "Xiaomi", "XIAOMI REDMI 13C 256GB 8GB RAM", 600000),
    ("https://www.falabella.com.co/falabella-co/product/128261646/Celular-Xiaomi-Redmi-13C-256GB-8RAM-Negro/128261647", "Xiaomi", "XIAOMI REDMI 13C 256GB 8GB RAM", 600000),
    ("https://www.falabella.com.co/falabella-co/product/128128076/Xiaomi-Redmi-Note-13-Pro-4g-256gb-8gb-Ram-Negro/128128077", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram", 879900),
    ("https://www.falabella.com.co/falabella-co/product/128807587/Xiaomi-Redmi-Note-13-Pro-4g-256gb-8gb-Ram-Purpura/128807589", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram", 879900),
    ("https://www.falabella.com.co/falabella-co/product/prod13530339/Celular-Xiaomi-Redmi-Note-13-Pro-256GB-8GB-Camara-Posterior-200MP+8MP+2MP-Camara-Frontal-16MP-Pantalla-6.67-Pulgadas/72853341", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram", 949900),
    ("https://www.falabella.com.co/falabella-co/product/128128195/Xiaomi-Redmi-Note-13-Pro-4g-256gb-8gb-Ram-Forest-Green/128128196", "Xiaomi", "Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram", 1299900),
    ("https://www.falabella.com.co/falabella-co/product/128931528/Celular-Xiaomi-Redmi-Note-13-8GB-256GB-Negro/128931529", "Xiaomi", "Xiaomi Redmi Note 13 8GB 256GB", 1079900),
    ("https://www.falabella.com.co/falabella-co/product/127891485/Celular-Xiaomi-Redmi-Note-13-256GB-8GB-RAM-Negro/127891486", "Xiaomi", "Xiaomi Redmi Note 13 8GB 256GB", 669900),
    
    # Samsung
    ("https://www.falabella.com.co/falabella-co/product/129586393/CELULAR-SAMSUNG-GALAXY-A55-5G-256GB-8-RAM-AZUL-NAVY/129586394", "Samsung", "SAMSUNG GALAXY A55", 2499900),
    ("https://www.falabella.com.co/falabella-co/product/129554425/Celular-Samsung-Galaxy-A55-5G-256Gb-8Ram-Azul-Oscuro/129554428", "Samsung", "SAMSUNG GALAXY A55", 2000000),
    ("https://www.falabella.com.co/falabella-co/product/prod13530496/Celular-Samsung-A15-256GB-8GB-RAM-Camara-Posterior-50-MP-Camara-Frontal-12MP-Pantalla-6.5-Pulgadas-+-MediaTek-MT6789V-CD-/72854633", "Samsung", "Samsung A15 256GB", 699900),
    ("https://www.falabella.com.co/falabella-co/product/132236410/CELULAR-SAMSUNG-GALAXY-A15-8+256GB-AZUL-NEGRO/132236411", "Samsung", "Samsung A15 256GB", 612900),
    ("https://www.falabella.com.co/falabella-co/product/128004219/CELULAR-SAMSUNG-GALAXY-A15-256GB-8-RAM-NEGRO-(Blue-Black)/128004220", "Samsung", "Samsung A15 256GB",599900 ),
    ("https://www.falabella.com.co/falabella-co/product/132236494/CELULAR-SAMSUNG-GALAXY-A15-8+256GB-AZUL/132236495", "Samsung", "Samsung A15 256GB", 612900),
    ("https://www.falabella.com.co/falabella-co/product/128906319/Celular-Samsung-Galaxy-A25-5G-256GB-8RAM+-Cargador/128906321", "Samsung", "Samsung Galaxy A25 5G 256GB 8RAM", 1235000),
    ("https://www.falabella.com.co/falabella-co/product/131931272/Celular-SAMSUNG-GALAXY-A25-256GB-+-8-RAM-AZUL-CLARO/131931273", "Samsung", "Samsung Galaxy A25 5G 256GB 8RAM", 1199900),
    ("https://www.falabella.com.co/falabella-co/product/72899217/Celular-Samsung-Galaxy-A35-5G-256GB-8GB-RAM-camara-posterior-50-MP-camara-frontal-13MP-pantalla-6.6-Pulgadas-+-Exynos-1380/72899217", "Samsung", "Samsung Galaxy A35 5G 256GB 8GB RAM", 969900),
    ("https://www.falabella.com.co/falabella-co/product/133744591/Celular-Samsung-Galaxy-A35-256Gb-5g-Lila/133744592", "Samsung", "Samsung Galaxy A35 5G 256GB 8GB RAM", 969900),
    ("https://www.falabella.com.co/falabella-co/product/128668464/Celular-Samsung-Galaxy-A05S-128Gb-Negro/128668465", "Samsung", "Samsung Galaxy A05S 128Gb", 449900),
    ("https://www.falabella.com.co/falabella-co/product/128035979/Samsung-S24-Ultra-256GB-12GB-Violet/128035980", "Samsung", "Samsung S24 Ultra 256GB 12GB", 7500000),
    ("https://www.falabella.com.co/falabella-co/product/128853893/Celular-Samsung-Galaxy-S24-Ultra-De-256GB12GB-RAM-Gris-5G/128853894", "Samsung", "Samsung S24 Ultra 256GB 12GB", 7500000),
    ("https://www.falabella.com.co/falabella-co/product/129574090/Samsung-Galaxy-S24-Ultra-512GB-12GB-Ram-5G-Violeta/129574091", "Samsung", "Samsung Galaxy S24 Ultra 512GB 12GB", 7499990),

    #Apple
    ("https://www.falabella.com.co/falabella-co/product/prod12000048/iPhone-13-128-GB-5G-4GB-RAM-Pantalla-6.1-Pulgadas-Chip-A15-Bionic/19604508", "Apple", "iPhone 13 128 GB 5G 4GB RAM", 2799900),
    ("https://www.falabella.com.co/falabella-co/product/prod13430667/iPhone-15-de-128GB-5G-6GB-RAM-Pantalla-6.1-pulgadas-Chip-A16-Bionic-Carga-Tipo-C-Dynamic-Island-Camara-48MP/72752105", "Apple", "iPhone 15 128GB 5G 6GB RAM", 3759900),
    ("https://www.falabella.com.co/falabella-co/product/prod13070047/iPhone-14-128GB-5G-6GB-RAM-Pantalla-6.1-Pulgadas-Chip-A15-Bionic-/60999138", "Apple", "iPhone 14 128GB 5G 6GB RAM", 3199900),
    ("https://www.falabella.com.co/falabella-co/product/prod10900027/iPhone-11-128-GB-4GB-RAM-Pantalla-6.1-Pulgadas-Chip-A13-Bionic-Camara-12-MP/9722720", "Apple", "iPhone 11 128 GB 4GB RAM", 2549900),
]

# Lista para almacenar los datos
F_data = []

# Iterar sobre cada producto
for url, marca, modelo, precio in F_productos:  # Añadir el precio al desempaquetar
    print(f"Procesando: {modelo} ({marca}) - Precio: {precio}")

    # Cargar la página del producto
    driver.get(url)

    # Esperar a que se cargue el contenedor de comentarios
    try:
        wait = WebDriverWait(driver, 30)  # Aumentar si es necesario para conexiones lentas
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='user-reviews-container']")))

        # Cargar todos los comentarios
        comentarios = []
        while True:
            try:
                ver_mas_btn = driver.find_element(By.ID, "more-comments-button")
                driver.execute_script("arguments[0].scrollIntoView();", ver_mas_btn)
                ver_mas_btn.click()

                # Esperar dinámicamente hasta que se carguen nuevos comentarios
                WebDriverWait(driver, 30).until(
                    lambda d: len(d.find_elements(By.CSS_SELECTOR, "div._review-text_16yc3_2")) > len(comentarios)
                )

                # Actualizar el número de comentarios cargados
                comentarios = driver.find_elements(By.CSS_SELECTOR, "div._review-text_16yc3_2")
                print(f"Comentarios cargados hasta ahora: {len(comentarios)}")  # Log de progreso

            except Exception:
                print("No hay más comentarios para cargar.")
                break

        # Extraer los comentarios y calificaciones
        F_reviews = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='review']")
        for review in F_reviews:
            try:
                # Extraer el comentario
                comentario_element = review.find_element(By.CSS_SELECTOR, "div._review-text_16yc3_2")
                comentario = comentario_element.text.strip()

                # Buscar las estrellas dentro del contenedor actual
                estrellas_llenas = review.find_elements(By.CSS_SELECTOR, "span._star-full_10acl_10")
                calificacion = len(estrellas_llenas)  # Contar las estrellas llenas

                print(f"Comentario: {comentario[:30]}... | Calificación: {calificacion} | Precio: {precio}")  # Log para depuración

                # Guardar en la lista
                F_data.append({
                    "comentario": comentario,
                    "calificacion": calificacion,
                    "marca": marca,
                    "modelo": modelo,
                    "precio": precio  # Añadir precio
                })

            except Exception as e:
                print(f"Error procesando una review: {e}")

    except Exception as e:
        print(f"Error procesando {modelo}: {e}")

# Crear un DataFrame con los resultados
F_df = pd.DataFrame(F_data)

# Ruta completa para guardar el archivo CSV
output_path = r"D:\Proyecto Web Scrapping\Webscrapping\Fallabella\comentarios_productos.csv"

# Exportar a CSV
F_df.to_csv(output_path, index=False, encoding="utf-8")
print(f"Se procesaron {len(F_data)} comentarios en total. Archivo guardado como '{output_path}'.")





# Cerrar el navegador
driver.quit()


Procesando: Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp (Xiaomi) - Precio: 1899900
Comentarios cargados hasta ahora: 7
Comentarios cargados hasta ahora: 25
Comentarios cargados hasta ahora: 33
Comentarios cargados hasta ahora: 41
Comentarios cargados hasta ahora: 47
Comentarios cargados hasta ahora: 56
Comentarios cargados hasta ahora: 58
No hay más comentarios para cargar.
Comentario: Si corresponde a la foto tiene... | Calificación: 5 | Precio: 1899900
Comentario: El tamaño es el apropiado y pa... | Calificación: 5 | Precio: 1899900
Comentario: Muy buen producto a un excelen... | Calificación: 5 | Precio: 1899900
Comentario: Me gusto mucho, y fue rápido l... | Calificación: 5 | Precio: 1899900
Comentario: Cumple lo que promete, es un l... | Calificación: 5 | Precio: 1899900
Comentario: Si es igual a la foto, me gust... | Calificación: 5 | Precio: 1899900
Comentario: Me gusto mucho, y fue rápido l... | Calificación: 5 | Precio: 1899900
Comentario: Cumple lo que promete, es un l... | Califi

In [11]:
F_df.groupby(['marca', 'modelo']).size()

marca    modelo                                            
Apple    iPhone 11 128 GB 4GB RAM                              1078
         iPhone 13 128 GB 5G 4GB RAM                            319
         iPhone 14 128GB 5G 6GB RAM                             632
         iPhone 15 128GB 5G 6GB RAM                             172
Samsung  SAMSUNG GALAXY A55                                    1696
         Samsung A15 256GB                                      539
         Samsung Galaxy A05S 128Gb                               15
         Samsung Galaxy A25 5G 256GB 8RAM                       238
         Samsung Galaxy A35 5G 256GB 8GB RAM                    305
         Samsung Galaxy S24 Ultra 512GB 12GB                    221
         Samsung S24 Ultra 256GB 12GB                          1334
Xiaomi   XIAOMI REDMI 13C 256GB 8GB RAM                         102
         Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp                 137
         Xiaomi Redmi Note 13 8GB 256GB                 

In [3]:
# Ruta del archivo CSV
file_path = r"D:\Documents\Scrapping Sentimientos Project\Project_repository\Webscrapping\Fallabella\comentarios_productos.csv"

# Leer el archivo CSV en un DataFrame
F_df = pd.read_csv(file_path)

# Reemplazar los valores faltantes en 'precio' cuando el modelo sea 'iPhone 14 128GB 5G 6GB RAM'
F_df.loc[(F_df['modelo'] == "iPhone 14 128GB 5G 6GB RAM") & (F_df['precio'].isna()), 'precio'] = 3199900.0

# Mostrar las primeras filas del DataFrame para verificar la importación
F_df

,comentario,calificacion,marca,modelo,precio
0,Si corresponde a la foto tiene un excelente re...,5,Xiaomi,Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp,1899900.0
1,El tamaño es el apropiado y para cargar fácilm...,5,Xiaomi,Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp,1899900.0
2,"Muy buen producto a un excelente precio, llego...",5,Xiaomi,Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp,1899900.0
3,"Me gusto mucho, y fue rápido la llegada",5,Xiaomi,Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp,1899900.0
4,"Cumple lo que promete, es un lindo celular",5,Xiaomi,Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp,1899900.0
...,...,...,...,...,...
7466,"El tamaño corresponde a la foto, es de buena c...",3,Apple,iPhone 14 128GB 5G 6GB RAM,3199900.0
7467,"Lindo color, igual a la foto",3,Apple,iPhone 14 128GB 5G 6GB RAM,3199900.0
7468,Horrible este teléfono no actualiza el softwar...,1,Apple,iPhone 14 128GB 5G 6GB RAM,3199900.0
7469,"Cumplió mis expectativas, el tamaño es el mism...",1,Apple,iPhone 14 128GB 5G 6GB RAM,3199900.0


In [4]:
# Diccionario de mapeo (modelo, precio) -> código
model_price_to_code = {
    ("iPhone 13 128 GB 5G 4GB RAM", 2799900.00): "FI1",
    ("iPhone 15 128GB 5G 6GB RAM", 3759900.00): "FI2",
    ("iPhone 14 128GB 5G 6GB RAM", 3199900.00): "FI3",
    ("iPhone 11 128 GB 4GB RAM", 2549900.00): "FI4",
    ("Apple iPhone 11 (128 GB) - Negro", 1754910.00): "MI5",
    ("Apple iPhone 13 (128 GB) - Azul", 2499900.00): "MI6",
    ("Apple iPhone 14 (128 GB) - Medianoche", 3169000.00): "MI7",
    ("Apple iPhone 15 (128 GB) - Negro", 3217900.00): "MI8",
    ("SAMSUNG GALAXY A55", 2499900.00): "FS1",
    ("SAMSUNG GALAXY A55", 2000000.00): "FS2",
    ("Samsung A15 256GB", 699900.00): "FS3",
    ("Samsung A15 256GB", 612900.00): "FS4",
    ("Samsung A15 256GB", 599900.00): "FS5",
    ("Samsung A15 256GB", 612900.00): "FS6",
    ("Samsung Galaxy A25 5G 256GB 8RAM", 1235000.00): "FS7",
    ("Samsung Galaxy A25 5G 256GB 8RAM", 1199900.00): "FS8",
    ("Samsung Galaxy A35 5G 256GB 8GB RAM", 969900.00): "FS9",
    ("Samsung Galaxy A35 5G 256GB 8GB RAM", 1299900.00): "FS10",
    ("Samsung Galaxy A05S 128Gb", 449900.00): "FS11",
    ("Samsung S24 Ultra 256GB 12GB", 7500000.00): "FS12",
    ("Samsung S24 Ultra 256GB 12GB", 7500000.00): "FS13",
    ("Samsung Galaxy S24 Ultra 512GB 12GB", 7499990.00): "FS14",
    ("Samsung A05s 4G Dual SIM 128 GB Negro 6 GB RAM", 431433.00): "MS15",
    ("Samsung Galaxy A15 4G Dual SIM 256 GB azul oscuro 8 GB RAM", 599900.00): "MS16",
    ("Samsung Galaxy A25 5G 5G 128 GB azul 6 GB RAM", 849900.00): "MS17",
    ("Samsung Galaxy A35 5g 256 GB Azul oscuro 8 GB RAM", 949900.00): "MS18",
    ("Samsung Galaxy A55 5G Dual SIM 256 GB azul oscuro 8 GB RAM", 1387900.00): "MS19",
    ("Samsung Galaxy S23 Ultra (eSIM) 5G Dual SIM 512 GB green 12 GB RAM", 4349900.00): "MS20",
    ("Samsung Galaxy S24 Ultra 5G Dual SIM 512 GB titanium black 12 GB RAM", 4529900.00): "MS21",
    ("Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp", 1899900.00): "FX1",
    ("Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp", 1449900.00): "FX2",
    ("Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp", 2199900.00): "FX3",
    ("Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp", 2199900.00): "FX4",
    ("Xiaomi Redmi Note 13 Pro Plus 5G 512Gb 12Ram 200Mp", 3099700.00): "FX5",
    ("XIAOMI REDMI 13C 256GB 8GB RAM", 600000.00): "FX6",
    ("XIAOMI REDMI 13C 256GB 8GB RAM", 600000.00): "FX7",
    ("Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram", 879900.00): "FX8",
    ("Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram", 879900.00): "FX9",
    ("Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram", 949900.00): "FX10",
    ("Xiaomi Redmi Note 13 Pro 4g 256gb 8gb Ram", 1299900.00): "FX11",
    ("Xiaomi Redmi Note 13 8GB 256GB", 1079900.00): "FX12",
    ("Xiaomi Redmi Note 13 8GB 256GB", 669900.00): "FX13",
    ("Xiaomi POCO Poco X6 Pro 5G Dual SIM 512 GB negro 12 GB RAM", 1304900.00): "MX14",
    ("Xiaomi Redmi 13C Dual SIM 256 GB midnight black 8 GB RAM", 479940.00): "MX15",
    ("Xiaomi Redmi A3 Dual SIM 64 GB midnight black 3 GB RAM", 299900.00): "MX16",
    ("Xiaomi Redmi Note 13 4G Dual SIM 256 GB negro 8 GB RAM", 617900.00): "MX17",
    ("Xiaomi Redmi Note 13 Pro 4G Dual SIM 256 GB púrpura 8 GB RAM", 890946.00): "MX18",
    ("Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB Negro 12 GB RAM", 1619940.00): "MX19",
}

# Crear una columna combinada en el DataFrame (modelo, precio)
F_df['model_price'] = list(zip(F_df['modelo'], F_df['precio']))

# Asignar los códigos basados en el diccionario
F_df['codigo'] = F_df['model_price'].map(model_price_to_code)

# Eliminar la columna auxiliar si no es necesaria
F_df.drop(columns=['model_price'], inplace=True)

# Verificar resultados
F_df

,comentario,calificacion,marca,modelo,precio,codigo
0,Si corresponde a la foto tiene un excelente re...,5,Xiaomi,Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp,1899900.0,FX1
1,El tamaño es el apropiado y para cargar fácilm...,5,Xiaomi,Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp,1899900.0,FX1
2,"Muy buen producto a un excelente precio, llego...",5,Xiaomi,Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp,1899900.0,FX1
3,"Me gusto mucho, y fue rápido la llegada",5,Xiaomi,Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp,1899900.0,FX1
4,"Cumple lo que promete, es un lindo celular",5,Xiaomi,Xiaomi Poco X6 Pro 5G 512Gb 12Ram 64Mp,1899900.0,FX1
...,...,...,...,...,...,...
7466,"El tamaño corresponde a la foto, es de buena c...",3,Apple,iPhone 14 128GB 5G 6GB RAM,3199900.0,FI3
7467,"Lindo color, igual a la foto",3,Apple,iPhone 14 128GB 5G 6GB RAM,3199900.0,FI3
7468,Horrible este teléfono no actualiza el softwar...,1,Apple,iPhone 14 128GB 5G 6GB RAM,3199900.0,FI3
7469,"Cumplió mis expectativas, el tamaño es el mism...",1,Apple,iPhone 14 128GB 5G 6GB RAM,3199900.0,FI3


In [22]:
############################ WEBSCRAPPING MERCADOLIBRE ############################

In [5]:
# Ruta de la carpeta que contiene los documentos
folder_path = r"D:\Documents\Scrapping Sentimientos Project\Project_repository\Webscrapping\Mercadolibre\HTMLs Mercadolibre"

# Lista para almacenar los datos de todos los archivos
all_data = []

# Iterar sobre todos los archivos en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith('.docx'):  # Verificar si el archivo es .docx
        file_path = os.path.join(folder_path, filename)

        # Leer el contenido del documento .docx
        doc = Document(file_path)
        html_content = "\n".join([paragraph.text for paragraph in doc.paragraphs])

        # Analizar el contenido HTML con BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extraer los comentarios
        comments = soup.find_all('p', class_='ui-review-capability-comments__comment__content')
        extracted_comments = [comment.get_text(strip=True) for comment in comments]

        # Extraer las calificaciones
        ratings = []
        rating_containers = soup.find_all('div', class_='ui-review-capability-comments__comment__rating')
        for container in rating_containers:
            stars = len(container.find_all('use', href='#poly_star_fill'))
            ratings.append(stars)

        # Asegurar que comentarios y calificaciones tengan el mismo tamaño
        if len(extracted_comments) > len(ratings):
            ratings.extend([None] * (len(extracted_comments) - len(ratings)))
        elif len(ratings) > len(extracted_comments):
            extracted_comments.extend([None] * (len(ratings) - len(extracted_comments)))

        # Agregar los datos al conjunto general
        for comment, rating in zip(extracted_comments, ratings):
            all_data.append({'Filename': filename, 'Comment': comment, 'Rating': rating})

# Crear el DataFrame con el nombre "mercadolibre_reviews"
mercadolibre_reviews = pd.DataFrame(all_data)

In [6]:
#DATA CLEANING
# Eliminar ".docx" si existe
mercadolibre_reviews['Filename'] = mercadolibre_reviews['Filename'].str.replace('.docx', '', regex=False)

# Extraer el modelo (todo lo que está antes del precio)
mercadolibre_reviews['Model'] = mercadolibre_reviews['Filename'].str.extract(r'^(.*)\s\$\d+')

# Eliminar la columna "Filename"
mercadolibre_reviews = mercadolibre_reviews.drop(columns=['Filename'])

# Reorganizar las columnas en el orden especificado
mercadolibre_reviews = mercadolibre_reviews[['Model', 'Comment', 'Rating']]

In [7]:
# Diccionario de mapeo del modelo al código
model_to_code = {
    "Apple iPhone 11 (128 GB) - Negro": "MI5",
    "Apple iPhone 13 (128 GB) - Azul": "MI6",
    "Apple iPhone 14 (128 GB) - Medianoche": "MI7",
    "Apple iPhone 15 (128 GB) - Negro": "MI8",
    "Samsung A05s 4G Dual SIM 128 GB Negro 6 GB RAM": "MS15",
    "Samsung Galaxy A15 4G Dual SIM 256 GB azul oscuro 8 GB RAM": "MS16",
    "Samsung Galaxy A25 5G 5G 128 GB azul 6 GB RAM": "MS17",
    "Samsung Galaxy A35 5g 256 GB Azul oscuro 8 GB RAM": "MS18",
    "Samsung Galaxy A55 5G Dual SIM 256 GB azul oscuro 8 GB RAM": "MS19",
    "Samsung Galaxy S23 Ultra (eSIM) 5G Dual SIM 512 GB green 12 GB RAM": "MS20",
    "Samsung Galaxy S24 Ultra 5G Dual SIM 512 GB titanium black 12 GB RAM": "MS21",
    "Xiaomi POCO Poco X6 Pro 5G Dual SIM 512 GB negro 12 GB RAM": "MX14",
    "Xiaomi Redmi 13C Dual SIM 256 GB midnight black 8 GB RAM": "MX15",
    "Xiaomi Redmi A3 Dual SIM 64 GB midnight black 3 GB RAM": "MX16",
    "Xiaomi Redmi Note 13 4G Dual SIM 256 GB negro 8 GB RAM": "MX17",
    "Xiaomi Redmi Note 13 Pro 4G Dual SIM 256 GB púrpura 8 GB RAM": "MX18",
    "Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB Negro 12 GB RAM": "MX19"
}

# Agregar la columna "Codigo" basada en el mapeo
mercadolibre_reviews['Codigo'] = mercadolibre_reviews['Model'].map(model_to_code)

# Guardar el DataFrame en un archivo CSV
mercadolibre_reviews.to_csv('all_comments_and_ratings.csv', index=False)

# Verificar el resultado
mercadolibre_reviews

,Model,Comment,Rating,Codigo
0,Apple iPhone 11 (128 GB) - Negro,"Es un dispositivo magnifico, se adapta a la ne...",5,MI5
1,Apple iPhone 11 (128 GB) - Negro,"Excelente precio para un equipo tan completo, ...",5,MI5
2,Apple iPhone 11 (128 GB) - Negro,No sabía que llegaba sin cargador.,4,MI5
3,Apple iPhone 11 (128 GB) - Negro,Ha sido excelente este producto. El celular ha...,5,MI5
4,Apple iPhone 11 (128 GB) - Negro,"Buen producto, totalmente nuevo.",5,MI5
...,...,...,...,...
2494,Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB N...,Es un buen teléfono gama media. Y muy bonito. ...,5,MX19
2495,Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB N...,,5,MX19
2496,Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB N...,Lo compre en el rush de un producto nuevo del ...,5,MX19
2497,Xiaomi Redmi Note 13 Pro+ 5G Dual SIM 512 GB N...,"Perfecto , su cámara es muy buena , el diseño ...",5,MX19


In [8]:
#Conteo de comentarios extraidos
# Agrupar por "Filename" y contar los comentarios únicos
grouped_comments = mercadolibre_reviews.groupby('Model')['Comment'].count()

print(grouped_comments)

Model
Apple iPhone 11 (128 GB) - Negro                                         83
Apple iPhone 13 (128 GB) - Azul                                         180
Apple iPhone 14 (128 GB) - Medianoche                                    50
Apple iPhone 15 (128 GB) - Negro                                        130
Samsung A05s 4G Dual SIM 128 GB Negro 6 GB RAM                           90
Samsung Galaxy A15 4G Dual SIM 256 GB azul oscuro 8 GB RAM              173
Samsung Galaxy A25 5G 5G 128 GB azul 6 GB RAM                            71
Samsung Galaxy A35 5g 256 GB Azul oscuro 8 GB RAM                       106
Samsung Galaxy A55 5G Dual SIM 256 GB azul oscuro 8 GB RAM              210
Samsung Galaxy S23 Ultra (eSIM) 5G Dual SIM 512 GB green 12 GB RAM      210
Samsung Galaxy S24 Ultra 5G Dual SIM 512 GB titanium black 12 GB RAM    111
Xiaomi POCO Poco X6 Pro 5G Dual SIM 512 GB negro 12 GB RAM              210
Xiaomi Redmi 13C Dual SIM 256 GB midnight black 8 GB RAM                210
Xiaomi

In [27]:
###################################### EXPORTAR DATAFRAMES A MYSQL #########################################

In [9]:
# Configura los detalles de la conexión
user = 'root'  # Cambia según tu configuración
password = 'Elefantes1507'  # Cambia según tu configuración
host = 'localhost'  # O 127.0.0.1
port = '3306'  # Puerto predeterminado de MySQL
database = 'sentimental_analysis_db'  # Nombre de tu base de datos

# Crear el motor de conexión
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}")

In [10]:
# Guardar los DataFrames en MySQL
# Asegúrate de que mercadolibre_reviews y F_df ya están definidos como DataFrames
mercadolibre_reviews.to_sql('mercadolibre_reviews', engine, if_exists='replace', index=False)
F_df.to_sql('f_reviews', engine, if_exists='replace', index=False)

print("DataFrames cargados exitosamente en MySQL.")

DataFrames cargados exitosamente en MySQL.


In [11]:
#GUARDAR MAESTRO DE PRODUCTOS EN SQL
# Ruta del archivo Excel
file_path = r"D:\Documents\Scrapping Sentimientos Project\Project_repository\Webscrapping\Fuentes de datos Web Scrapping.xlsx"

# Leer el archivo Excel
maestro_productos = pd.read_excel(file_path)

# Eliminar espacios al inicio y al final de cada valor en todas las celdas
maestro_productos = maestro_productos.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Nombre de la tabla donde se guardarán los datos
table_name = 'webscrapping_sources'

# Guardar el DataFrame en MySQL
maestro_productos.to_sql(table_name, engine, if_exists='replace', index=False)

print(f"Tabla '{table_name}' creada y datos cargados exitosamente en la base de datos '{database}'.")

Tabla 'webscrapping_sources' creada y datos cargados exitosamente en la base de datos 'sentimental_analysis_db'.


C:\Users\ASUS\AppData\Local\Temp\ipykernel_9112\2712947780.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  maestro_productos = maestro_productos.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
#EXPORTAR TABLA FINAL CON SOLO LOS MODELOS QUE TIENEN MAS DE 200 COMENTARIOS
# Cargar la tabla completa
combined_reviews = pd.read_sql("SELECT * FROM combined_reviews", con=engine)

# Filtrar en Python
filtered_reviews = combined_reviews.groupby('Modelo estandar').filter(lambda x: x['comentario'].nunique() >= 200)

# Guardar la tabla filtrada nuevamente en MySQL
filtered_reviews.to_sql('filtered_reviews', con=engine, if_exists='replace', index=False)

9620

In [ ]:
# Ruta donde deseas IMPORTAR el archivo CSVSQL Queries/filtered_reviews.csv
output_path = r"D:\Documents\Scrapping Sentimientos Project\Project_repository\SQL Queries\filtered_reviews.csv"

# Exportar el DataFrame como un archivo CSV
#filtered_reviews.to_csv(output_path, index=False, encoding='utf-8')

print(f"Archivo CSV guardado exitosamente en: {output_path}")

Archivo CSV guardado exitosamente en: D:\Documents\Scrapping Sentimientos Project\Project_repository\SQL Queries\filtered_reviews.csv


In [ ]:
###################################### IMPORTAR DATAFRAME FINAL FROM SQL #########################################

In [ ]:
# Configura los detalles de la conexión
user = 'root'  # Cambia según tu configuración
password = 'Elefantes1507'  # Cambia según tu configuración
host = 'localhost'  # O 127.0.0.1
port = '3306'  # Puerto predeterminado de MySQL
database = 'sentimental_analysis_db'  # Nombre de tu base de datos

# Crear el motor de conexión
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}")


# Cargar la tabla completa
df_as = pd.read_sql("SELECT * FROM FILTERED_reviews", con=engine)

In [ ]:
###################################### IMPORTAR DATAFRAME FINAL FROM CSV #########################################

In [ ]:
df_as = pd.read_csv("SQL Queries/filtered_reviews.csv")
print(df_as.head())

                                          comentario  calificacion codigo  \
0  Si corresponde a la foto tiene un excelente re...             5    FX1   
1  El tamaño es el apropiado y para cargar fácilm...             5    FX1   
2  Muy buen producto a un excelente precio, llego...             5    FX1   
3            Me gusto mucho, y fue rápido la llegada             5    FX1   
4         Cumple lo que promete, es un lindo celular             5    FX1   

                                              Enlace  Sitio Web   Marca  \
0  https://www.falabella.com.co/falabella-co/prod...  falabella  Xiaomi   
1  https://www.falabella.com.co/falabella-co/prod...  falabella  Xiaomi   
2  https://www.falabella.com.co/falabella-co/prod...  falabella  Xiaomi   
3  https://www.falabella.com.co/falabella-co/prod...  falabella  Xiaomi   
4  https://www.falabella.com.co/falabella-co/prod...  falabella  Xiaomi   

      Modelo estandar   Precio  
0  Xiaomi Poco X6 Pro  1899900  
1  Xiaomi Poco X6 Pr